In [9]:
import random
import gym
import numpy as np
import tensorflow as tf
tf.compat.v1.reset_default_graph()
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

class agent(object):
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.states = []
        self.rewards = []
        self.labels = []
        self.gamma = 0.99
        self.learning_rate = 0.0001
     
        # parameters to optimize
        self.X = tf.placeholder(tf.float32, [1, self.state_size])
        self.label = tf.placeholder(tf.float32, [1, self.action_size])
        self.G = tf.placeholder(tf.float32, [])
        self.W1 = tf.Variable(tf.random_normal([self.state_size, 20], stddev=0.1))
        self.b1 = tf.Variable(tf.zeros([20]))
        self.L1 = tf.nn.relu(tf.matmul(self.X, self.W1) + self.b1)
        self.W2 = tf.Variable(tf.random_normal([20, self.action_size], stddev=0.1))
        self.b2 = tf.Variable(tf.zeros([self.action_size]))
        self.output = tf.nn.softmax(tf.matmul(self.L1, self.W2) + self.b2)
        
        
        self.W1_v = tf.Variable(tf.random_normal([self.state_size, 20], stddev=0.1))
        self.b1_v = tf.Variable(tf.zeros([20]))
        self.L1_v = tf.nn.relu(tf.matmul(self.X, self.W1_v) + self.b1_v)
        self.W2_v = tf.Variable(tf.random_normal([20, self.action_size], stddev=0.1))
        self.b2_v = tf.Variable(tf.zeros([self.action_size]))
        self.V = tf.matmul(self.L1_v, self.W2_v) + self.b2_v
        
              
        self.cross_entropy_loss = -tf.reduce_sum(self.label * tf.log(self.output)) * (self.G-self.V)
        self.train_optimizer1 = tf.train.AdamOptimizer(self.learning_rate).minimize(self.cross_entropy_loss)

        self.mean_square_loss = tf.reduce_mean((self.G - self.V) ** 2)
        self.train_optimizer2 = tf.train.AdamOptimizer(learning_rate=0.001).minimize(self.mean_square_loss)



        # initialize tensorflow session
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())


    def store(self, state, action, reward):
        self.states.append(state)
        self.rewards.append(reward)
        label = np.zeros(self.action_size)
        label[action] = 1
        self.labels.append(label)

    def _G(self, t):
        G = 0
        for tau in range(t, len(self.rewards)):
            G = G + self.gamma**(tau - t) * self.rewards[tau]
        return G

    def train(self):
        for t in range(len(self.states)):
            state = self.states[t]
            label = self.labels[t]
            G = self._G(t)
            self.sess.run(self.train_optimizer1, feed_dict={self.X: state.reshape(1, -1), self.G: G, self.label: label.reshape(1, -1)})
            self.sess.run(self.train_optimizer2, feed_dict={self.X: state.reshape(1, -1), self.G: G, self.label: label.reshape(1, -1)})
        self.states = []
        self.labels = []
        self.rewards = []
        
    def see_variables(self):
        print(self.sess.run(self.W1)) 
    
    def act(self, state):
        # run forward propagation to get softmax probabilities
        prob_weights = self.sess.run(self.output, feed_dict = {self.X: state.reshape(1, -1)})
        action = np.random.choice(range(len(prob_weights.ravel())), p=prob_weights.ravel())
        # action = np.random.choice(np.arange(len(prob_weights[0])), prob_weights[0])
   
        return action

env = gym.make('LunarLander-v2')
state = env.reset()
score = 0
total_score = 0
episode = 0
state_size = 8
action_size = env.action_space.n
g = agent(state_size, action_size)

while episode < 3000:  # episode loop
    action = g.act(state)
    next_state, reward, done, info = env.step(action)  # take a random action
    score = score + reward
    total_score = total_score + reward
    g.store(state, action, reward)
    state = next_state
#    g.see_variables()

    if done:
        episode = episode + 1
        g.train()
        if episode % 100 == 0:
            print('Episode: {} Score: {} Total score: {}'.format(episode, score, total_score))
        score = 0
        state = env.reset()
        
# TEST     
episode = 0
state = env.reset()     
while episode < 10:  # episode loop
    env.render()
    action = g.act(state)
    next_state, reward, done, info = env.step(action)  # take a random action
    state = next_state

    if done:
        episode = episode + 1
        state = env.reset()
env.close()       


Episode: 100 Score: -316.05092465458785 Total score: -16888.70399054631
Episode: 200 Score: -83.92534801341736 Total score: -33579.11620086123
Episode: 300 Score: -106.30482274967586 Total score: -47504.51260124024
Episode: 400 Score: -126.90775032043065 Total score: -61583.03862462868
Episode: 500 Score: -117.70716062021148 Total score: -73718.6732615154
Episode: 600 Score: -151.2576996795596 Total score: -85522.20856020862
Episode: 700 Score: -137.2077034121664 Total score: -98522.24823435103
Episode: 800 Score: -43.61075766774573 Total score: -110215.53413895276
Episode: 900 Score: -60.56203454943346 Total score: -121255.0245269523
Episode: 1000 Score: -79.80899093953343 Total score: -132137.75533398375
Episode: 1100 Score: -59.264265101594475 Total score: -141333.12067482714
Episode: 1200 Score: -74.18954024137847 Total score: -149428.71373013905
Episode: 1300 Score: -215.64777875764474 Total score: -156079.90426005246
Episode: 1400 Score: -12.376281257937464 Total score: -161522.5

KeyboardInterrupt: 